### Scratch space for CNN model for Chessy

In [1]:
import json
import random
import matplotlib.pyplot as plot
import numpy as np
import os
import jsonlines
import ast
import time, os, fnmatch, shutil
from collections import defaultdict
import matplotlib
import requests

import math
import random
import copy

from collections import namedtuple, deque

import numpy as np

In [2]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F


device  = torch.device("cuda:0" if torch.cuda.is_available() else "cuda:1")
device

device(type='cuda', index=0)

In [72]:
args = {
    'in_channels_1':1,
    'in_channels_2':64,
    'in_channels_l':128,
    'out_channels_1':64,
    'out_channels_2':128,
    'out_channels_l':16,
    'kernel_1_size':(6,1),
    'kernel_2_size':(1,1),
    'kernel_l_size':(1,1),
    'stride_1_size':(6,1),
    'stride_2_size':(6,1),
    'stride_l_size':(1,1),
    'reshape_size':(8,48),
    'action_size':88,
}

In [78]:
s1_ = '010010010011010100010101010110010111011000011001011010011011000000000000011110011111000000100001000000000000011100000000011101000000000000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000101000000000000000001000010000011000100001101000110000111001000001001001010001011001100000000001110001111010000'
s1  = np.array(list(map(int, s1_)))

In [79]:
s1

array([0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,

In [85]:
s2d = torch.tensor(s1.astype(np.float32)).to(device)
s2d.shape

torch.Size([384])

In [86]:
class ConvNet(torch.nn.Module):
    
    def __init__(self):
        
        super(ConvNet, self).__init__()
        
        self.conv1  = nn.Conv2d(in_channels=args['in_channels_1'],out_channels=args['out_channels_1'], kernel_size=args['kernel_1_size'], stride=args['stride_1_size']).to(device)
        self.conv2  = nn.Conv2d(in_channels=args['in_channels_2'],out_channels=args['out_channels_2'], kernel_size=args['kernel_2_size'], stride=args['stride_2_size']).to(device)
        self.linear = nn.Linear(args['in_channels_l']*args['out_channels_l'], args['action_size']).to(device)

        
    def forward(self, x):
        x   = x.reshape(args['reshape_size']).unsqueeze(0).unsqueeze(0)
        out    = F.relu(self.conv1(x))
        out    = F.relu(self.conv2(out))
        logits = self.linear(out.view(-1, args['in_channels_l']*args['out_channels_l']))
        probs  = torch.mean(F.softmax(logits, dim=1), axis=0)
        
        return probs

In [87]:
model = ConvNet()

In [88]:
model(s2d)

tensor([0.0120, 0.0106, 0.0107, 0.0112, 0.0108, 0.0104, 0.0112, 0.0116, 0.0112,
        0.0124, 0.0121, 0.0119, 0.0118, 0.0126, 0.0115, 0.0108, 0.0117, 0.0112,
        0.0115, 0.0114, 0.0112, 0.0121, 0.0115, 0.0116, 0.0107, 0.0116, 0.0119,
        0.0111, 0.0112, 0.0107, 0.0114, 0.0108, 0.0111, 0.0103, 0.0116, 0.0110,
        0.0111, 0.0113, 0.0124, 0.0119, 0.0114, 0.0106, 0.0120, 0.0109, 0.0111,
        0.0121, 0.0112, 0.0108, 0.0124, 0.0114, 0.0110, 0.0113, 0.0120, 0.0104,
        0.0122, 0.0112, 0.0108, 0.0113, 0.0113, 0.0111, 0.0105, 0.0114, 0.0114,
        0.0116, 0.0123, 0.0109, 0.0108, 0.0115, 0.0118, 0.0120, 0.0119, 0.0117,
        0.0110, 0.0115, 0.0108, 0.0110, 0.0112, 0.0114, 0.0105, 0.0119, 0.0116,
        0.0115, 0.0109, 0.0112, 0.0119, 0.0116, 0.0108, 0.0117],
       device='cuda:0', grad_fn=<MeanBackward1>)

### Setup TensorBoard

In [96]:
import torch
from torch.utils.tensorboard import SummaryWriter, summary
writer = SummaryWriter()

ImportError: TensorBoard logging requires TensorBoard version 1.15 or above

In [94]:
!pip install --upgrade torch
!pip install tensorboard

Requirement already up-to-date: torch in /home/ubuntu/anaconda3/lib/python3.7/site-packages (1.6.0)
ERROR: Error checking for conflicts.
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/lib/python3.7/site-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/home/ubuntu/anaconda3/lib/python3.7/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/lib/python3.7/site-packages/pip/_vendor/pkg_resources/__init__.py", line 3012, in _parsed_pkg_info
    return self._pkg_info
  File "/home/ubuntu/anaconda3/lib/python3.7/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _pkg_info

During handling of the above exce